# Student 2938740
## Part 4 Optimisation Assigment

In [1]:
# Import libraries
import numpy as np
from random import choice
from time import perf_counter

In [2]:
def initialiseSolution():
    
    # Create day vectors for reference, use tuples to preserve data integrity
    monthDays = 31
    isFriday = (2, 9, 16, 23)
    isSaturday = (3, 10, 17, 24)
    isMonday = (5, 12, 19, 26)
    
    # Create holiday types
    holTypes = ('F7', 'S7', 'M7', 'F3', 'M4')

    # Create accommodations (C = Caravan, L = Lodge)
    C1, C2, C3, C4, C5 = [0]*monthDays, [0]*monthDays, [0]*monthDays, [0]*monthDays, [0]*monthDays
    L1, L2, L3, L4, L5 = [0]*monthDays, [0]*monthDays, [0]*monthDays, [0]*monthDays, [0]*monthDays
    accommodations = dict({'caravans':{'C1':C1, 'C2':C2, 'C3':C3, 'C4':C4, 'C5':C5},
                                   'lodges':{'L1':L1, 'L2':L2, 'L3':L3, 'L4':L4, 'L5':L5}})

    # Create price structure
    prices = dict({'caravans':{'F7':250, 'S7':500, 'M7':600, 'F3':350, 'M4':400},
                  'lodges':{'F7':350, 'S7':600, 'M7':700, 'F3':450, 'M4':500}})
    
    return isFriday, isSaturday, isMonday, holTypes, accommodations, prices

In [3]:
def getContiguousDays(acc):
    
    # Calculate the longest stretch of contiguous days and return length + index
    maxCount, maxIdx, count = -1, 0, 0

    for idx, i in enumerate(acc):
        if (acc[idx] == 1) :
            count = 0
        else:
            count += 1

        # Update values    
        maxCount, maxIdx = (maxCount, count)[count > maxCount], (maxIdx, idx-(count-1))[count > maxCount]
    
    # Debug print
    #print(maxCount, maxIdx)
        
    return (maxCount, maxIdx)

In [11]:
def generateBookingPlan(epochs, steps, unit):
    
    # Initialise placeholders for best performance and core variables
    bestRev, bestPenalty, bestNumBookings, bestDaysAvail, bestAdjRev, bestBookings, bestBkgVec = 0, 0, 0, 0, 0, [], []
    bestEpoch = 0
    penPerc = 0.05  # Penalty percentage for wasted days
    
    # Perform epochs
    for epoch in range(epochs):

        # Initialise variables
        isFriday, isSaturday, isMonday, holTypes, accommodations, prices = initialiseSolution()

        # Determine current accommodation variables for price and booking vector
        accommType = ('caravans', 'lodges')[unit[0] == 'L']
        bkgVec = accommodations[accommType][unit]
        
        # Reset variables for totals
        currRev, daysBooked, daysAvail, numBookings, currBookings = 0, 0, 31, 0, []

        # Perform random search
        for i in range(steps):

            #print(f"Iteration {i+1} in progress...")

            # Calculate remaining contiguous days and index value
            maxDays, maxIdx = getContiguousDays(bkgVec)
            #print(maxDays, maxIdx)

            # If greater than 6 days all options are available to book
            if (maxDays > 6):
                # Select holiday at random: pos1 = Day, pos2 = Duration
                hol = choice(holTypes)
                holDay, holDur = hol[0], int(hol[1])
                holPrice = prices[accommType][hol]
                # Use start day to randomly select arrival date 
                if (holDay == 'F'):
                    holStart = choice(isFriday)
                elif (holDay == 'S'):
                    holStart = choice(isSaturday)
                elif (holDay == 'M'):
                    holStart = choice(isMonday)

                """ To minimise wasted days look to fill gaps with specific options """
            # Are less than 7 days available? Check contiguous days for F3 and M4 bookings
            elif ((maxDays > 2) & (maxDays < 7)):
                if ((maxIdx+1) in isFriday):
                    hol = 'F3'
                    holDay, holDur = hol[0], int(hol[1])
                    holPrice = prices[accommType][hol]
                    holStart = maxIdx+1
                elif ((maxIdx+1) in isMonday):
                    hol = 'M4'
                    holDay, holDur = hol[0], int(hol[1])
                    holPrice = prices[accommType][hol]
                    holStart = maxIdx+1

            # If less than 3 contiguous day no more space to book
            else:
                #print(f"Bookings completed on iteration {i+1}/{steps}!")
                break

            # Debug print summary of choices made before passing to bookings block
            #print(f"hol: {hol}, holDay: {holDay}, holDur: {holDur}, holStart: {holStart}")


            # Check unit booking vector for availability and make booking
            if ((np.sum(bkgVec[(holStart-1):(holStart+holDur-1)]) == 0) & (holStart+holDur < 32)):
                bkgVec[:] = [1 if idx in range(holStart-1, holStart+holDur-1) else b for idx, b in enumerate(bkgVec)]
                numBookings += 1
                daysBooked += holDur
                daysAvail -= holDur
                currRev += holPrice
                currBookings.append((hol, holStart, holPrice))
            #else:
            #    print(f"Sorry these dates are booked. Current booking vector:\n{accomm}\n\n")

        # Calculate wastage penalty if more than 3 days remain (you cannot book less than 3 days)
        if (maxDays < 3):
            penalty = 0
        else:
            penalty = currRev * penPerc * daysAvail
        adjRev = currRev - penalty

        # Optimal bookings results (long form because of number of variables for readability)
        if (currRev > bestRev):
            bestRev = currRev
            bestPenalty = int(penalty)
            bestNumBookings = numBookings
            bestDaysAvail = daysAvail
            bestAdjRev = int(adjRev)
            bestBookings[:] = currBookings
            bestBkgVec = bkgVec
            bestEpoch = epoch + 1
            
        # Summarise optimal bookings search for current epoch
        #print(f"SUMMARY FOR EPOCH {epoch+1} of {epochs}")
        #print(f"Revenue generated: {currRev}, Wastage penalty: {int(penalty)}")
        #print(f"Bookings: {numBookings}, Days unbooked: {daysAvail}\nAdjusted revenue: {int(adjRev)}")
        #print(f"{'-'*58}")
        #print(f"Optimal booking summary (Booking type, Arrival day, Price)\n{currBookings}\nBookings vector: {bkgVec}\n\n")


    # Summarise optimal bookings search over all epochs
    print(f"\033[1mAccommodation {unit}\033[0m : Optimal summary from epoch {bestEpoch}")
    print(f"Revenue generated: {bestRev}, Wastage penalty: {bestPenalty}")
    print(f"Bookings: {bestNumBookings}, Days unbooked: {bestDaysAvail}\nAdjusted revenue: {bestAdjRev}")
    print(f"{'-'*58}")
    print(f"Optimal booking summary (Booking type, Arrival day, Price)\n{bestBookings}\nBookings vector: {bestBkgVec}\n\n")

    return

### Execution block

In [13]:
# Optimiser function is: generateBookingPlan(epochs, steps, unit)  # unit = accommodation unit

tik = perf_counter()
_, _, _, _, accommodations, _ = initialiseSolution()

for val in accommodations:
    print(f"\033[1mCommencing optimisation run for accommodation type: {val}\033[0m\n{'-'*60}")
    for key in accommodations[val]:
        generateBookingPlan(5, 50, key)

tok = perf_counter()
elapsed = tok-tik
print(f"\n10 accommodations each with 5 epochs of 50 steps completed in {round(elapsed, 2)} seconds.")

Commencing optimisation run for accommodation type: caravans
------------------------------------------------------------
Accommodation C1 : Optimal summary from epoch 1
Revenue generated: 2850, Wastage penalty: 0
Bookings: 7, Days unbooked: 3
Adjusted revenue: 2850
----------------------------------------------------------
Optimal booking summary (Booking type, Arrival day, Price)
[('M7', 19, 600), ('F3', 2, 350), ('M4', 5, 400), ('M4', 26, 400), ('F3', 16, 350), ('M4', 12, 400), ('F3', 9, 350)]
Bookings vector: [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0]


Accommodation C2 : Optimal summary from epoch 1
Revenue generated: 2150, Wastage penalty: 645
Bookings: 5, Days unbooked: 6
Adjusted revenue: 1505
----------------------------------------------------------
Optimal booking summary (Booking type, Arrival day, Price)
[('F3', 9, 350), ('S7', 24, 500), ('S7', 17, 500), ('M4', 5, 400), ('M4', 12, 400)]
Bookings vector: [0, 0, 0, 0, 1, 1, 